In [1]:
import pandas as pd
import numpy as np

import random

import datetime

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import TruncatedSVD

from catboost import CatBoostRegressor

from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

from tqdm import tqdm

seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
df = pd.read_csv("./data/youtube/youtube_train.csv", index_col="id")
df_test = pd.read_csv("./data/youtube/youtube_test.csv", index_col="id")
df.head()

,Channel,Subtitles,PublishDate,Category,ViewCount
id,,,,,
0,toplesofficial,('Я миллениал и я не занимаюсь сексом так част...,2017-12-31T01:35:44-08:00,Education,3019
1,postnauka,"(""сервитуты это одно из прямых наследие римско...",2014-09-04T08:24:47-07:00,Science & Technology,25
2,NaukaPRO,('[музыка] существуют сенсоры измеряющие магни...,2023-11-23T21:00:08-08:00,Science & Technology,7
3,postnauka,('в наше время только ленивые не говорит о том...,2014-06-23T03:49:31-07:00,Science & Technology,5
4,user-rb8ux1no6j,('дорогие друзья это честный рекламный ролик н...,2020-10-09T01:30:04-07:00,Education,10


In [3]:
min_date = pd.to_datetime(datetime.datetime.strptime('01012010', "%d%m%Y").date())
df["PublishDate"] = df["PublishDate"].apply(lambda x: datetime.datetime.strptime(x[:-6], "%Y-%m-%dT%H:%M:%S"))
df["PublishDateDays"] = (df["PublishDate"] - min_date).dt.days
df["PublishDateYear"] = df["PublishDate"].dt.year
df["PublishDateMonth"] = df["PublishDate"].dt.month
df["PublishDateDay"] = df["PublishDate"].dt.day
df["PublishDateDay"] = df["PublishDate"].dt.day_of_week
df["PublishDateDay"] = df["PublishDate"].dt.day_of_year

In [4]:
df_test["PublishDate"] = df_test["PublishDate"].apply(lambda x: datetime.datetime.strptime(x[:-6], "%Y-%m-%dT%H:%M:%S"))
df_test["PublishDateDays"] = (df_test["PublishDate"] - min_date).dt.days
df_test["PublishDateYear"] = df_test["PublishDate"].dt.year
df_test["PublishDateMonth"] = df_test["PublishDate"].dt.month
df_test["PublishDateDay"] = df_test["PublishDate"].dt.day
df_test["PublishDateDay"] = df_test["PublishDate"].dt.day_of_week
df_test["PublishDateDay"] = df_test["PublishDate"].dt.day_of_year

In [5]:
X = df.drop(columns=["ViewCount", "Subtitles"])
y = df["ViewCount"]

# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=seed)
# X_train = X_train.reset_index()
# X_val = X_val.reset_index()

In [6]:
X_test = df_test.drop(columns=["Subtitles"])

In [7]:
model = SentenceTransformer("BAAI/bge-m3", device=device)

In [8]:
model.max_seq_length = 512

In [9]:
embs = model.encode(X["Subtitles"].tolist(), batch_size=8, show_progress_bar=True, normalize_embeddings=True, device=device)
# embs_val = model.encode(X_val["Subtitles"].tolist(), batch_size=8, show_progress_bar=True, normalize_embeddings=True, device=device)
embs_test = model.encode(X_test["Subtitles"].tolist(), batch_size=8, show_progress_bar=True, normalize_embeddings=True, device=device)

Batches:   0%|          | 0/732 [00:00<?, ?it/s]

Batches:   0%|          | 0/250 [00:00<?, ?it/s]

In [10]:
n_components = 16
svd = TruncatedSVD(n_components=n_components, random_state=seed)
svd.fit(embs.tolist())
embs_svd = svd.transform(embs.tolist())
embs_test_svd = svd.transform(embs_test.tolist())

In [11]:
X = pd.concat([X, pd.DataFrame(embs_svd.tolist(), columns=[f"emb_{i}" for i in range(n_components)])], axis=1)
X_test = pd.concat([X_test, pd.DataFrame(embs_test_svd.tolist(), columns=[f"emb_{i}" for i in range(n_components)], index=X_test.index)], axis=1)

In [7]:
cat_features = list()

for col_name, dtype in X.dtypes.to_dict().items():
    if dtype == "object" and col_name != "Subtitles":
        cat_features.append(col_name)

cat_features

['Channel', 'Category']

In [8]:
n_splits = 100
kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
val_scores = list()
test_preds = np.zeros((X_test.shape[0], n_splits))

for i, (train_index, val_index) in enumerate(tqdm(kf.split(X))):
    print(f"=== FOLD {i + 1} ===")

    X_train = X.iloc[train_index]
    X_val = X.iloc[val_index]
    y_train = y.iloc[train_index]
    y_val = y.iloc[val_index]

    model = CatBoostRegressor(
        iterations=1500,
        cat_features=cat_features,
        # text_features=["Subtitles"],
        eval_metric="MAE",
        learning_rate=0.1,
        verbose=250,
        early_stopping_rounds=200,
        use_best_model=True,
        random_seed=seed,
        # depth=8,
    )
    model.fit(X_train, y_train, eval_set=(X_val, y_val))

    preds = model.predict(X_val)
    test_preds[:, i] = model.predict(X_test)
    
    val_scores.append(mean_absolute_error(y_val, preds))

sum(val_scores) / len(val_scores)

0it [00:00, ?it/s]

=== FOLD 1 ===
0:	learn: 468.6390245	test: 380.0739796	best: 380.0739796 (0)	total: 50.7ms	remaining: 1m 16s
250:	learn: 174.0726355	test: 193.9261651	best: 193.3972115 (244)	total: 397ms	remaining: 1.97s


1it [00:00,  1.21it/s]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 192.8422942
bestIteration = 282

Shrink model to first 283 iterations.
=== FOLD 2 ===
0:	learn: 475.3306644	test: 366.8837654	best: 366.8837654 (0)	total: 2.01ms	remaining: 3.01s


2it [00:01,  1.72it/s]

...

99it [01:05,  1.65it/s]

=== FOLD 100 ===
0:	learn: 474.1986060	test: 404.6392725	best: 404.6392725 (0)	total: 1.81ms	remaining: 2.71s


100it [01:05,  1.52it/s]

250:	learn: 164.4408445	test: 177.2570605	best: 165.5944357 (51)	total: 342ms	remaining: 1.7s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 165.5944357
bestIteration = 51

Shrink model to first 52 iterations.


175.2835665566453

In [9]:
test_preds = test_preds.mean(axis=1)
test_preds.shape

(2000,)

In [10]:
sample_submission = pd.read_csv("./data/youtube/youtube_sample_submission.csv", index_col="id")
sample_submission["ViewCount"] = test_preds
sample_submission.to_csv("./submissions/youtube/sub6.csv")

In [3]:
# min_date = pd.to_datetime(datetime.datetime.strptime('01012010', "%d%m%Y").date())
# df["PublishDate"] = df["PublishDate"].apply(lambda x: datetime.datetime.strptime(x[:-6], "%Y-%m-%dT%H:%M:%S"))
# df["PublishDateDays"] = (df["PublishDate"] - min_date).dt.days
# df["PublishDateYear"] = df["PublishDate"].dt.year
# df["PublishDateMonth"] = df["PublishDate"].dt.month
# df["PublishDateDay"] = df["PublishDate"].dt.day

In [4]:
# df_test["PublishDate"] = df_test["PublishDate"].apply(lambda x: datetime.datetime.strptime(x[:-6], "%Y-%m-%dT%H:%M:%S"))
# df_test["PublishDateDays"] = (df_test["PublishDate"] - min_date).dt.days
# df_test["PublishDateYear"] = df_test["PublishDate"].dt.year
# df_test["PublishDateMonth"] = df_test["PublishDate"].dt.month
# df_test["PublishDateDay"] = df_test["PublishDate"].dt.day

In [5]:
# df.isna().sum()

Channel             0
Subtitles           0
PublishDate         0
Category            0
ViewCount           0
PublishDateDays     0
PublishDateYear     0
PublishDateMonth    0
PublishDateDay      0
dtype: int64

In [6]:
# X = df.drop(columns=["ViewCount", "Subtitles"])
# y = df["ViewCount"]

In [7]:
# X_test = df_test.drop(columns=["Subtitles"])

In [8]:
# cat_features = list()

# for col_name, dtype in X.dtypes.to_dict().items():
#     if dtype == "object" and col_name != "Subtitles":
#         cat_features.append(col_name)

# cat_features

['Channel', 'Category']

In [9]:
# n_splits = 5
# kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
# val_scores = list()
# test_preds = np.zeros((X_test.shape[0], n_splits))

# for i, (train_index, val_index) in enumerate(tqdm(kf.split(X))):
#     print(f"=== FOLD {i + 1} ===")

#     X_train = X.iloc[train_index]
#     X_val = X.iloc[val_index]
#     y_train = y.iloc[train_index]
#     y_val = y.iloc[val_index]

#     model = CatBoostRegressor(
#         iterations=500,
#         cat_features=cat_features,
#         # text_features=["Subtitles"],
#         eval_metric="MAE",
#         # learning_rate=0.07,
#         verbose=250,
#         early_stopping_rounds=200,
#         use_best_model=True,
#         random_seed=seed,
#         # depth=8,
#     )
#     model.fit(X_train, y_train, eval_set=(X_val, y_val))

#     preds = model.predict(X_val)
#     test_preds[:, i] = model.predict(X_test)
    
#     val_scores.append(mean_absolute_error(y_val, preds))

# sum(val_scores) / len(val_scores)

0it [00:00, ?it/s]

=== FOLD 1 ===
Learning rate set to 0.099023
0:	learn: 474.6785850	test: 455.6021404	best: 455.6021404 (0)	total: 50.6ms	remaining: 25.3s
250:	learn: 155.7788338	test: 201.1125630	best: 201.0719664 (249)	total: 367ms	remaining: 364ms


1it [00:00,  1.27it/s]

499:	learn: 133.6669297	test: 203.8270172	best: 199.9057328 (361)	total: 702ms	remaining: 0us

bestTest = 199.9057328
bestIteration = 361

Shrink model to first 362 iterations.
=== FOLD 2 ===
Learning rate set to 0.099023
0:	learn: 454.7121915	test: 512.4080574	best: 512.4080574 (0)	total: 1.77ms	remaining: 886ms


2it [00:01,  1.79it/s]

...

4it [00:02,  2.04it/s]

250:	learn: 167.2952062	test: 178.5031845	best: 175.0625532 (142)	total: 312ms	remaining: 310ms
Stopped by overfitting detector  (200 iterations wait)

bestTest = 175.0625532
bestIteration = 142

Shrink model to first 143 iterations.
=== FOLD 5 ===
Learning rate set to 0.099027
0:	learn: 457.6805473	test: 487.2536470	best: 487.2536470 (0)	total: 1.57ms	remaining: 784ms


5it [00:02,  2.00it/s]

250:	learn: 151.6950821	test: 195.0853830	best: 190.8870395 (89)	total: 317ms	remaining: 315ms
Stopped by overfitting detector  (200 iterations wait)

bestTest = 190.8870395
bestIteration = 89

Shrink model to first 90 iterations.


190.4859490048181